In [1]:
!pip install substrate-interface

     |████████████████████████████████| 186 kB 739 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 625 kB/s eta 0:00:01


In [2]:
import os
import contextlib

In [3]:
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="https://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [4]:
# Function to make extrinsic calls
def make_call(call_module, call_function, call_params, keypair):
    call = substrate.compose_call(
        call_module=call_module,
        call_function=call_function,
        call_params=call_params
    )

    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair)

    try:
        receipt = substrate.submit_extrinsic(extrinsic, wait_for_inclusion=True)
        print("Extrinsic '{}' sent and included in block '{}'".format(receipt.extrinsic_hash, receipt.block_hash))

    except Exception as e:
        print("Failed to send: {}".format(e))
    return receipt

In [5]:
alice = Keypair.create_from_uri('//Alice')
bob = Keypair.create_from_uri('//Bob')

In [6]:
make_call("Msa", "create", {}, alice)
make_call("Msa", "create", {}, bob)

KeyboardInterrupt: 

In [ ]:
# Create never before used wallet
new_wallet = Keypair.create_from_uri('//BrandNewWallet4', crypto_type=substrateinterface.KeypairType.SR25519)

In [ ]:
# Give wallet some tokens to it can make MSA
# Must be at least 1 unit
one_unit = 1000000000000
receipt = make_call("Balances", "transfer", {"dest": new_wallet.ss58_address, "value": 10 * one_unit}, alice)
receipt.error_message

In [ ]:
msa_key = substrate.query(
    module='Msa',
    storage_function='KeyInfoOf',
    params=[new_wallet.ss58_address]
)

if msa_key == None:
    make_call("Msa", "create", {}, new_wallet)
    msa_key = substrate.query(
        module='Msa',
        storage_function='KeyInfoOf',
        params=[new_wallet.ss58_address]
    )
    
msa_id = msa_key['msa_id'].decode()
msa_id

In [ ]:
# encode payload with 
payload_raw = { "msa_id": msa_id, "nonce": substrate.get_account_nonce(new_wallet.ss58_address) + 1 }
add_provider_payload = substrate.encode_scale(type_string='scale_info::8', value=payload_raw['msa_id']) + substrate.encode_scale(type_string='scale_info::4', value=payload_raw['nonce'])
add_provider_payload1 = "".encode() + add_provider_payload.data
add_provider_payload2 = substrate.encode_scale(type_string='str', value="<Bytes>") + add_provider_payload + substrate.encode_scale(type_string='str', value="</Bytes>")
add_provider_payload3 = "<Bytes>".encode() + add_provider_payload.data + "</Bytes>".encode()

In [ ]:
# This helps find what scale_info to use for parameters
# substrate.metadata_decoder

In [ ]:
signature = new_wallet.sign(add_provider_payload)
signature1 = new_wallet.sign(add_provider_payload1)
signature2 = new_wallet.sign(add_provider_payload2)
signature3 = new_wallet.sign(add_provider_payload3)

In [ ]:
# signature = new_wallet.sign(blake2b(add_provider_payload.data, digest_size=32).digest())

In [ ]:
for i, key in enumerate(["0x" + new_wallet.public_key.hex(), new_wallet.ss58_address]):
    for j, s in enumerate([signature, signature1, signature2, signature3]):
        print(i, j)
        call_params = {
            "key": key,
            "proof": {"Sr25519": "0x" + s.hex()},
            "add_key_payload": payload_raw
        }

        # mute function as I made it output a lot of info
        with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
            receipt = make_call("Msa", "add_key_to_msa", call_params, new_wallet)
        print(receipt.error_message)
        print()

In [ ]:
add_provider_payload3 = "<Bytes>".encode() + add_provider_payload.data + "</Bytes>".encode()
signature3 = new_wallet.sign(add_provider_payload3)

key = "0x" + new_wallet.public_key.hex()
s = signature3

call_params = {
    "key": key,
    "proof": {"Sr25519": "0x" + s.hex()},
    "add_key_payload": payload_raw
}

# mute function as I made it output a lot of info
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    receipt = make_call("Msa", "add_key_to_msa", call_params, new_wallet)
print(receipt.error_message)
print()

# Attempt to delegate

In [ ]:
payload_raw = { "authorized_msa_id": msa_id, "permission": 0 }
add_provider_payload = substrate.encode_scale(type_string='scale_info::8', value=payload_raw['authorized_msa_id']) + substrate.encode_scale(type_string='scale_info::2', value=payload_raw['permission'])
add_provider_payload3 = "<Bytes>".encode() + add_provider_payload.data + "</Bytes>".encode()
signature3 = alice.sign(add_provider_payload3)


key = "0x" + alice.public_key.hex()
s = signature3

call_params = {
    "provider_key": key,
    "proof": {"Sr25519": "0x" + s.hex()},
    "add_provider_payload": payload_raw
}

# mute function as I made it output a lot of info
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    receipt = make_call("Msa", "add_provider_to_msa", call_params, new_wallet)
print(receipt.error_message)
print()

# Attempt to create with delegation

In [ ]:
msa_key = substrate.query(
    module='Msa',
    storage_function='KeyInfoOf',
    params=[new_wallet.ss58_address]
)

In [ ]:
msa_key

In [ ]:
msa_key = substrate.query(
    module='Msa',
    storage_function='KeyInfoOf',
    params=[alice.ss58_address]
)

if msa_key == None:
    make_call("Msa", "create", {}, new_wallet)
    msa_key = substrate.query(
        module='Msa',
        storage_function='KeyInfoOf',
        params=[alice.ss58_address]
    )
    
provider_msa_id = msa_key['msa_id'].decode()

nex_msa_id = substrate.query(
    module='Msa',
    storage_function='MsaIdentifier',
    params=[]
).value + 1

payload_raw = { "authorized_msa_id": provider_msa_id, "permission": 0 }
add_provider_payload = substrate.(type_string='scale_info::8', value=payload_raw['authorized_msa_id']) + \
    substrate.encode_scale(type_string='scale_info::2', value=paylencode_scaleoad_raw['permission'])
add_provider_payload3 = "<Bytes>".encode() + add_provider_payload.data + "</Bytes>".encode()
signature3 = new_wallet.sign(add_provider_payload3)


key = "0x" + new_wallet.public_key.hex()
s = signature3

call_params = {
    "delegator_key": key,
    "proof": {"Sr25519": "0x" + s.hex()},
    "add_provider_payload": payload_raw
}

# mute function as I made it output a lot of info
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    receipt = make_call("Msa", "create_sponsored_account_with_delegation", call_params, alice)
print(receipt.error_message)
print()